Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [52]:
#Connect to data folder of NerLTR-DTA
root_path = '/data/'
import os
os.chdir(root_path)
!pwd

/content/gdrive/My Drive/KI JKU/Seminar_PracticalW_MasterThesis/MasterThesis/NerLTR/data


In [53]:

import numpy as np
import pandas as pd
import linecache
import math
from collections import Counter
import pickle
from posixpath import split

#Load Data

In [54]:
path_score_KIBA = 'RankLibEval/data_KIBA/test_score_KIBA.txt'
path_test_KIBA = 'RankLibEval/data_KIBA/test_KIBA.txt'

In [55]:
path_score_KIBA = 'RankLibEval/data_KIBA2/test_score_KIBA2.txt'
path_test_KIBA = 'RankLibEval/data_KIBA2/test_KIBA_Ner.txt'

In [56]:
path_test_KIBA = 'RankLibEval/data_KIBA3/test_KIBA_Ner.txt'
path_score_KIBA_1 = 'RankLibEval/data_KIBA3/test_score_KIBA_1.txt'
path_score_KIBA_2 = 'RankLibEval/data_KIBA3/test_score_KIBA_3.txt'
path_score_KIBA_3 = 'RankLibEval/data_KIBA3/test_score_KIBA_3.txt'

In [57]:
#split1
path_split1_score = 'RankLibEval/data_KIBA2/split1/val_score_1.txt'
path_split1_val = 'RankLibEval/data_KIBA2/split1/val1.txt'
path_split1_score_test = 'RankLibEval/data_KIBA2/split1/test_score_1.txt'
#split2
path_split2_score = 'RankLibEval/data_KIBA2/split2/val_score_2.txt'
path_split2_val = 'RankLibEval/data_KIBA2/split2/val2.txt'
path_split2_score_test = 'RankLibEval/data_KIBA2/split2/test_score_2.txt'
#split1
path_split3_score = 'RankLibEval/data_KIBA2/split3/val_score_3.txt'
path_split3_val = 'RankLibEval/data_KIBA2/split3/val3.txt'
path_split3_score_test = 'RankLibEval/data_KIBA2/split3/test_score_3.txt'
#split1
path_split4_score = 'RankLibEval/data_KIBA2/split4/val_score_4.txt'
path_split4_val = 'RankLibEval/data_KIBA2/split4/val4.txt'
path_split4_score_test = 'RankLibEval/data_KIBA2/split4/test_score_4.txt'
#split1
path_split5_score = 'RankLibEval/data_KIBA2/split5/val_score_5.txt'
path_split5_val = 'RankLibEval/data_KIBA2/split5/val5.txt'
path_split5_score_test = 'RankLibEval/data_KIBA2/split5/test_score_5.txt'

##Get Predictions

In [58]:
#KIBA
pred_KIBA = []
with open(path_score_KIBA_2) as f:
  for line in f:
    split=line.split()
    pred_score = float(split[2])
    pred_KIBA.append(pred_score)



In [59]:
print(f'Len Prd KIBA: {len(pred_KIBA)}')

Len Prd KIBA: 19709


In [60]:
def get_pred_score(input_path):
  pred = []
  with open(input_path) as f:
    for line in f:
      split=line.split()
      pred_score = float(split[2])
      pred.append(pred_score)
  return pred


##Get Truth

In [63]:
#KIBA
y_KIBA = []
with open(path_test_KIBA) as f:
  for line in f:
    split=line.split()
    pred_score = float(split[0])
    y_KIBA.append(pred_score)


In [64]:
print(f'Len Truth KIBA: {len(y_KIBA)}')

Len Truth KIBA: 19709


In [65]:
def get_truth(input_path):
  truth = []
  with open(input_path) as f:
    for line in f:
      split=line.split()
      pred_score = float(split[0])
      truth.append(pred_score)
  return truth


#Get IDs

In [68]:
#Affinities
path_test_aff_KIBA = 'affi/test_aff_KIBA.pkl'

In [69]:
#test
test_affi_KIBA = pd.read_pickle(path_test_aff_KIBA)
y_test_KIBA_ID = test_affi_KIBA['target_ID'].tolist()


##Metrics

In [70]:
# install lifelines
!pip install -q lifelines
from lifelines.utils import concordance_index
from sklearn.metrics import mean_squared_error, r2_score

In [71]:
def r_squared_error(y_obs,y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    y_pred_mean = [np.mean(y_pred) for y in y_pred]

    mult = sum((y_pred - y_pred_mean) * (y_obs - y_obs_mean))
    mult = mult * mult

    y_obs_sq = sum((y_obs - y_obs_mean)*(y_obs - y_obs_mean))
    y_pred_sq = sum((y_pred - y_pred_mean) * (y_pred - y_pred_mean) )

    return mult / float(y_obs_sq * y_pred_sq)
def squared_error_zero(y_obs,y_pred):
    k = get_k(y_obs,y_pred)

    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    upp = sum((y_obs - (k*y_pred)) * (y_obs - (k* y_pred)))
    down= sum((y_obs - y_obs_mean)*(y_obs - y_obs_mean))

    return 1 - (upp / float(down))

def get_k(y_obs,y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)

    return sum(y_obs*y_pred) / float(sum(y_pred*y_pred))
def get_rm2(ys_orig,ys_line):
    r2 = r_squared_error(ys_orig, ys_line)
    r02 = squared_error_zero(ys_orig, ys_line)
    return r2 * (1 - np.sqrt(np.absolute((r2*r2)-(r02*r02))))

import torch
def concordance_index_taskwise(predictions, labels, target_ids):
    CIs = list()
    target_id_list = list()
    predictions = torch.tensor(predictions)
    labels = torch.tensor(labels)
    target_ids = torch.tensor(target_ids)

    for target_idx in torch.unique(target_ids):
        rows = torch.where(target_ids == target_idx)
        preds = predictions[rows]
        y = labels[rows]

        if torch.unique(y).shape[0] >= 2:
            ci = concordance_index(y, preds)
            CIs.append(ci)
            target_id_list.append(target_idx.item())
        else:
            CIs.append(np.nan)
            target_id_list.append(target_idx.item())
    return np.nanmean(CIs), CIs, target_id_list


In [72]:
print("KIBA")
ci, CIs, _ = concordance_index_taskwise(pred_KIBA, y_KIBA, y_test_KIBA_ID)
print(f'Concordance index New:\t {ci}')
#print(f'Concordance index Old:\t {concordance_index(y_KIBA, pred_KIBA)}')
print(f"MSE\t\t：{mean_squared_error(y_KIBA, pred_KIBA)}")
print(f"rm2\t\t：{get_rm2(y_KIBA, pred_KIBA)}")

KIBA
Concordance index New:	 0.8444831598332931
MSE		：0.18979772274042064
rm2		：0.704343001916262


In [73]:
CIs

[0.6756756756756757,
 0.8653846153846154,
 0.8174904942965779,
 0.8489010989010989,
 0.8750959815715382,
 0.8792071435580414,
 0.7417218543046358,
 0.8648270345930814,
 0.9542097488921714,
 0.72544080604534,
 0.8756906077348067,
 0.7984097287184284,
 0.8330719247143177,
 0.9787234042553191,
 0.7800891530460624,
 0.8333333333333334,
 0.7716383772721801,
 0.8530600029926679,
 0.8999374609130707,
 0.9333333333333333,
 0.8837423454118514,
 0.7936354019746121,
 0.8455677655677656,
 0.7960291871712201,
 0.8895088342209824,
 0.771657455132584,
 0.8571428571428571,
 0.8795063332250731,
 0.8433861201676758,
 0.9566113998185267,
 0.9619016606968415,
 0.8549551135363492,
 0.859338061465721,
 0.8919763806877388,
 0.8787755102040816,
 0.7881996974281392,
 0.8856874044454899,
 1.0,
 0.8942262186464742,
 0.8580743618201998,
 0.8130728412928123,
 0.8023642732049037,
 0.8530670470756063,
 0.797971259509721,
 0.8562895291637386,
 1.0,
 0.8228404099560761,
 0.7739914808318717,
 0.8156410256410257,
 0.724

Davis


NameError: ignored

#Val

In [ ]:
print("Split1")
print(f'Concordance index:\t {concordance_index(truth_split1, pred_split1)}')
print(f"MSE\t\t：{mean_squared_error(truth_split1, pred_split1)}")
print(f"rm2\t\t：{get_rm2(truth_split1, pred_split1)}")

In [ ]:
print("Split2")
print(f'Concordance index:\t {concordance_index(truth_split2, pred_split2)}')
print(f"MSE\t\t：{mean_squared_error(truth_split2, pred_split2)}")
print(f"rm2\t\t：{get_rm2(truth_split2, pred_split2)}")

In [ ]:
print("Split3")
print(f'Concordance index:\t {concordance_index(truth_split3, pred_split3)}')
print(f"MSE\t\t：{mean_squared_error(truth_split3, pred_split3)}")
print(f"rm2\t\t：{get_rm2(truth_split3, pred_split3)}")

In [ ]:
print("Split4")
print(f'Concordance index:\t {concordance_index(truth_split4, pred_split4)}')
print(f"MSE\t\t：{mean_squared_error(truth_split4, pred_split4)}")
print(f"rm2\t\t：{get_rm2(truth_split4, pred_split4)}")

In [ ]:
print("Split5")
print(f'Concordance index:\t {concordance_index(truth_split5, pred_split5)}')
print(f"MSE\t\t：{mean_squared_error(truth_split5, pred_split5)}")
print(f"rm2\t\t：{get_rm2(truth_split5, pred_split5)}")

#Test

In [ ]:
print("Split1")
print(f'Concordance index:\t {concordance_index(truth_test, pred_split1_test)}')
print(f"MSE\t\t：{mean_squared_error(truth_test, pred_split1_test)}")
print(f"rm2\t\t：{get_rm2(truth_test, pred_split1_test)}")

In [ ]:
print("Split2")
print(f'Concordance index:\t {concordance_index(truth_test, pred_split2_test)}')
print(f"MSE\t\t：{mean_squared_error(truth_test, pred_split2_test)}")
print(f"rm2\t\t：{get_rm2(truth_test, pred_split2_test)}")

In [ ]:
print("Split3")
print(f'Concordance index:\t {concordance_index(truth_test, pred_split3_test)}')
print(f"MSE\t\t：{mean_squared_error(truth_test, pred_split3_test)}")
print(f"rm2\t\t：{get_rm2(truth_test, pred_split3_test)}")

In [ ]:
print("Split4")
print(f'Concordance index:\t {concordance_index(truth_test, pred_split4_test)}')
print(f"MSE\t\t：{mean_squared_error(truth_test, pred_split4_test)}")
print(f"rm2\t\t：{get_rm2(truth_test, pred_split4_test)}")

In [ ]:
print("Split5")
print(f'Concordance index:\t {concordance_index(truth_test, pred_split5_test)}')
print(f"MSE\t\t：{mean_squared_error(truth_test, pred_split5_test)}")
print(f"rm2\t\t：{get_rm2(truth_test, pred_split5_test)}")